In [14]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [15]:
if 'data' not in pn.state.cache.keys():
    df = pd.read_csv('https://raw.githubusercontent.com/WillBallentine/cov_dashboard/main/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time_-_ARCHIVED.csv')
    pn.state.cache['data'] = df.copy()
    
else:
    df = pn.state.cache['data']
    
df

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,year,month
0,2020-01-22,ND,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Not agree,2020,1
1,2020-01-22,MP,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,1/24/2020 0:00,Agree,Agree,2020,1
2,2020-01-22,OH,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Agree,2020,1
3,2020-01-22,GA,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Agree,2020,1
4,2020-01-22,PW,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,NaN,NaN,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60055,2022-10-18,UT,1041651,1041651.0,0.0,0,0.0,5034,4754.0,280.0,0,0.0,10/19/2022 13:15,Agree,Agree,2022,10
60056,2022-10-18,GU,58703,NaN,NaN,0,0.0,401,NaN,NaN,0,0.0,10/19/2022 13:15,Not agree,Not agree,2022,10
60057,2022-10-18,PA,3280190,2698966.0,581224.0,0,0.0,47442,NaN,NaN,0,0.0,10/19/2022 13:15,Agree,Not agree,2022,10
60058,2022-10-18,IN,1930022,NaN,NaN,0,0.0,24816,23745.0,1071.0,0,0.0,10/19/2022 13:15,Not agree,Agree,2022,10


In [13]:
#Variable set for radial buttons
df['submission_date'] = pd.to_datetime(df['submission_date'])
df['year'] = pd.DatetimeIndex(df['submission_date']).year
df['month'] = pd.DatetimeIndex(df['submission_date']).month

months = set(pd.DatetimeIndex(df['submission_date']).month)
years = set(pd.DatetimeIndex(df['submission_date']).year)
year = []
for i in years:
    year.append(i)
    
month = []
for i in months:
    month.append(i)


In [ ]:
df = df.fillna(0)

In [ ]:
idf = df.interactive()

In [ ]:
#Define panel widgets
state_select = pn.widgets.Select(name="State Select", options=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY',], inline=False)
state_select

In [ ]:
#define radial buttons for charts
month_case_select = pn.widgets.RadioButtonGroup(
    name='Month', 
    options=month,
    button_type='success'
)

year_case_select = pn.widgets.RadioButtonGroup(
    name='Year',
    options=year,
    button_type='success'
)

month_death_select = pn.widgets.RadioButtonGroup(
    name='Month', 
    options=month,
    button_type='success'
)

year_death_select = pn.widgets.RadioButtonGroup(
    name='Year',
    options=year,
    button_type='success'
)

In [ ]:
tot_case_pipeline = (idf[((idf.state) == state_select) & ((idf.month) == month_case_select) & ((idf.year) == year_case_select)])

In [ ]:
tot_deaths_pipeline = (idf[((idf.state) == state_select) & ((idf.month) == month_death_select) & ((idf.year) == year_death_select)])

In [ ]:
tot_case_pipeline

In [ ]:
tot_deaths_pipeline

In [ ]:
tot_plot = tot_case_pipeline.hvplot(kind='scatter', x='submission_date', by='state', y='tot_cases', title="Total Cases by state", color='red', height= 500, width=500, size=60)
tot_plot

In [ ]:
deaths_scatterplot = tot_deaths_pipeline.hvplot(x='submission_date', y='tot_death', size=60, kind="scatter", height = 500, width=500, title="Total Deaths by State")
deaths_scatterplot

In [ ]:
avg_death_pipeline = (idf[(idf.state) == state_select])

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Covid Cases by State', 
    sidebar=[pn.pane.Markdown("# Covid-19 Cases by State Reports"), 
             pn.pane.Markdown("#### Covid, while largely under conrol, had a massive impact on our States over the last three years."), 
             pn.pane.PNG('covid.png', width=200, height=100),
             pn.pane.Markdown("## Select State"),   
             state_select],
    main=[pn.Row(pn.Column(month_case_select, year_case_select,
                           tot_plot.panel(width=900), margin=(0,25)),
                 (pn.Column(month_death_select, year_death_select,
                 deaths_scatterplot.panel(width=900), margin=(0,25)))), 
],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();